# Install required libraries for solution 

In [ ]:
!pip install gym==0.21.0

In [ ]:
!pip install gym-retro

In [ ]:
!pip install opencv-contrib-python --user

# Import all required libraries

In [1]:
# Import retro to play AlienSoldier using a ROM
import retro
# Import time to slow down game
import time
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

# Import environment base class for a wrapper 
from gym import Env 
# Import the space shapes for the environment
from gym.spaces import MultiBinary, Box
# Import numpy to calculate frame delta 
import numpy as np
# Import opencv for grayscaling
import cv2
from datetime import date


## Create game environment wrapper

In [2]:
#Adapted class called StreetFighter from https://github.com/nicknochnack/StreetFighterRL/blob/main/StreetFighter-Tutorial.ipynb
#Made changes to the __init__ function to allow the user the option to record the game play , changed it work for Alien Soldier
#enviroment , added logic to set game state name.

# Create custom environment 
class AlienSoldier(Env): 
    def __init__(self,record_results=False,record_path=''):
        super().__init__()
        
        # Specify action space and observation space 
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        
        # Startup and instance of the game 
        if record_results:
            self.game = retro.make(game='AlienSoldier-Genesis', use_restricted_actions=retro.Actions.FILTERED, record=record_path)
        else:
            self.game = retro.make(game='AlienSoldier-Genesis', use_restricted_actions=retro.Actions.FILTERED)
            
        #set the game state name so able to access which level the game is on
        self.statename = self.game.statename
    
    def reset(self):
        # Return the first frame 
        obs = self.game.reset()
        obs = self.preprocess(obs) 
        self.previous_frame = obs 
        
        # reset attribute which holds the score delta
        self.score = 0 
        return obs
    
    def preprocess(self, observation): 
        # Grayscaling 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Resize 
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        # Add the channels value
        channels = np.reshape(resize, (84,84,1))
        return channels 
    
    def step(self, action): 
        # Take a step 
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs) 
        
        # Frame delta 
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs 
        
        # Reshape the reward function
        reward = info['score'] - self.score 
        self.score = info['score'] 
        
        return frame_delta, reward, done, info
    
    def render(self, *args, **kwargs):
        self.game.render()
        
    def close(self):
        self.game.close()

## Helper functions

In [3]:
# A function to retrun the level from state name in the retro enviroment. The state name contains alot string data 
# around it e.g DefaultSettings.Level1.state only interested in the level int as would like to log which level agnet ends on
def getLevelFromStateName(state_name):
    
    # initializing substrings
    start_str = "evel"
    end_str = ".state"
 
    # getting index of substrings
    start_index = state_name.find(start_str)
    end_index = state_name.find(end_str)
    
    # sub string the level from state name as we only want level as number 
    level = state_name[start_index + len(start_str): end_index]

    # convert the level to int and retrun it 
    return int(level)

In [4]:
# Function to start game environment and have the base line agent play the number games passed to function  
def baselineAgentPlayGame(LOG_DIR,model_name,RECORD_PATH,RecordGamePlay,NumerOfGamesToPlay):
    # Starts up the game environment
    env = AlienSoldier(record_results=RecordGamePlay,record_path=RECORD_PATH)

    # Setup writer which will log the scalar values from end of agent game play 
    writer = tf.summary.create_file_writer(LOG_DIR,name=model_name)

    print("------------------------------------")
    # Reset game to starting state
    obs = env.reset()
    # Set done flag to flase this indicates if agent game is done
    done = False

    # Loop over number games to play   
    for game in range(NumerOfGamesToPlay): 
        print("Game Num" ,game + 1)

        #Check if agents game has ended 
        while not done: 
            # Render the game frame 
            env.render()
            #Set the values from step function
            obs, reward, done, info = env.step(env.action_space.sample())    

            #Check if game is done, if true then proceed to log scalar values to writer to be logged 
            if done: 
                print("Game over")
                print("states",info)

                # Log the values to writer 
                with writer.as_default():
                    tf.summary.scalar("score", info["score"], step=game + 1)
                    tf.summary.scalar("time", info["time"], step=game + 1)
                    tf.summary.scalar("health", info["health"], step=game + 1)
                    tf.summary.scalar("level", getLevelFromStateName(env.statename), step=game + 1)
                    writer.flush()
                print("------------------------------------")
        # Reset obs so game can restart
        obs = env.reset()
        done = False  

## Setup paths to log and record agent game play

In [5]:
# Setup the paths to log to , path to record the game play to
LOG_DIR = './logs_final/final_results_base_line'
model_name = 'final_results_base_line_' + date.today().strftime('%Y-%m-%d')
RECORD_PATH= './RecordAgentsGamePlay/final_results_base_line'

## Start game enviroment and start agent playing game for required amount of games 

In [6]:
baselineAgentPlayGame(LOG_DIR,model_name,RECORD_PATH,True,1)

------------------------------------
Game Num 1


C:\Users\Tiger\.conda\envs\game\lib\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


Game over
states {'health': 0, 'score': 500, 'time': 125}
------------------------------------


## Convert game play to mp4

Update the below cmd to path where your game play is saved 

In [ ]:
%run -m retro.scripts.playback_movie RecordAgentsGamePlay/final_results_base_line/AlienSoldier-Genesis-DefaultSettings.Level1-000000.bk2